Python notebook for exploratory data analysis of the City of Chicago Parking Ticket Data

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings("ignore")

import datetime

In [4]:
# read the dataframe
df_CCPT = pd.read_csv('./data/parking_tickets_frac.csv')

In [5]:
df_CCPT.head(5)

,Unnamed: 0,ticket_number,issue_date,violation_location,license_plate_number,license_plate_state,license_plate_type,zipcode,violation_code,violation_description,...,fine_level2_amount,current_amount_due,total_payments,ticket_queue,ticket_queue_date,notice_level,hearing_disposition,notice_number,officer,address
0,19727593,9185717028,2014-08-05 21:26:00,1852 N MAUD,1e55730db26522e18e6cc4e37db1d5e49a1e9740ef56b5...,MI,PAS,NaN,0964090E,RESIDENTIAL PERMIT PARKING,...,150,0.0,75.0,Paid,2014-08-18 00:00:00,NaN,NaN,0,1432,"1800 n maud, chicago, il"
1,19367998,64352592,2014-06-14 09:41:00,4057 N SUNNYSIDE,e2ceaa3ed4fba9fa9ddeb146a1218c43eedac433ebacef...,IL,PAS,605212454,0964040B,STREET CLEANING,...,120,0.0,60.0,Paid,2014-08-23 00:00:00,NaN,NaN,5181940070,18937,"4000 n sunnyside, chicago, il"
2,5681589,9177655732,2009-01-07 19:56:00,3324 N HALSTED ST,c822dc8d016c735a2d95f33cd9fda4c89519c7aa8c1073...,IL,PAS,606473722,0964200B,PARK OUTSIDE METERED SPACE,...,100,0.0,50.0,Paid,2009-02-05 00:00:00,VIOL,NaN,5105450510,1030,"3300 n halsted st, chicago, il"
3,22647956,9188902856,2015-10-24 14:55:00,1953 N LACROSSE AV,0dc52cb43957b09dfc5514e2080d5c1e2a8de3747fe144...,IL,PAS,606512031,0964190A,EXP. METER NON-CENTRAL BUSINESS DISTRICT,...,100,0.0,50.0,Paid,2015-10-29 00:00:00,NaN,NaN,5131145590,1408,"1900 n lacrosse av, chicago, il"
4,1161221,9058219394,2007-05-30 17:27:00,1151 W WAVELAND,c99f5ac386092c0274cc2b247e1a411baf8f3a1587965b...,IL,PAS,600351014,0964090E,RESIDENTIAL PERMIT PARKING,...,100,0.0,50.0,Paid,2007-06-26 00:00:00,VIOL,NaN,5084345240,678,"1100 w waveland, chicago, il"


In [ ]:
hd = {'Unique Entry': df_CCPT.nunique(axis = 0),
        'Nan Entry': df_CCPT.isnull().any()}
pd.DataFrame(data = hd, index = df_CCPT.columns.values)

# potential columns to drop: ticket_number,violation_location (since its normalized to address),license_plate_number,
# zipcode, unit, notice_number  

In [6]:
to_drop = ['Unnamed: 0','ticket_number','violation_location','license_plate_number',
           'zipcode','unit','unit_description','notice_number']
df_CCPT.drop(labels=to_drop ,axis = 1,inplace = True)

In [7]:
row2drop= ['license_plate_state','license_plate_type']
df_CCPT.dropna(axis = 0, how = 'any', subset = row2drop, inplace = True)

In [ ]:
df_CCPT['license_plate_state'].isnull().any()

In [8]:
df_CCPT['issue_date'] = pd.to_datetime(df_CCPT['issue_date'])
df_CCPT['issue_date'].head(5)


0   2014-08-05 21:26:00
1   2014-06-14 09:41:00
2   2009-01-07 19:56:00
3   2015-10-24 14:55:00
4   2007-05-30 17:27:00
Name: issue_date, dtype: datetime64[ns]

In [9]:
# Getting the data from year 2015 to 2017
mask16 = ((df_CCPT['issue_date'] >= pd.Timestamp(2016,1,1,0)) & (df_CCPT['issue_date'] <pd.Timestamp(2018,1,1,0)))
df_CCPT16 = df_CCPT.loc[mask16]

In [ ]:
wd = {'weekday': df_CCPT16['issue_date'].dt.weekday,
        'hourofday': df_CCPT16['issue_date'].dt.hour}
df_date = pd.DataFrame(data = wd, index = df_CCPT16.index)

In [ ]:
plt.figure(figsize=(9,15))
plt.title('Days and Hours of Violations', fontweight='bold',fontsize = 16);
plt.hist2d(df_date['weekday'],df_date['hourofday'],[np.linspace(-0.5,6.5,num=8),np.linspace(-0.5,23.5,num=25)]);
plt.xticks(np.arange(7), ['Mon','Tue','Wed','Thu','Fri','Sat','Sun'], fontweight='bold');
hours_indx = [(datetime.time(i).strftime('%I %p')) for i in range(24)]
plt.yticks(np.arange(24), hours_indx, fontweight='bold');

In [ ]:
df_CCPT16['license_plate_state'].value_counts().head(10)

In [ ]:
x_ticks = df_CCPT16['license_plate_state'].value_counts().index
heights = df_CCPT16['license_plate_state'].value_counts()
y_pos = np.arange(len(x_ticks))
fig = plt.figure(figsize=(15,14)) 
# set horizontal bars
plt.barh(y_pos, heights)
 
# set names on the y-axis
plt.yticks(y_pos, x_ticks)
plt.gca().invert_yaxis() 
# plot figure
plt.show()

In [ ]:
plate_state = df_CCPT16['license_plate_state'].value_counts().index
ticket_status = df_CCPT16['ticket_queue'].value_counts().index
# Here I categorize the ticket status in to 4:
# Paid, Dissmissed, Likely to be Guilty (Notice), Unlikely to pay (the rest: Define, Court, Bankrupcy,Hearing Req)
count_by_state = np.zeros([2,4])


mask_instate = (df_CCPT16['license_plate_state'] == 'IL')
count_by_state[0,0] = np.count_nonzero(df_CCPT16.loc[mask_instate]['ticket_queue'] == 'Paid')
count_by_state[0,1] = np.count_nonzero(df_CCPT16.loc[mask_instate]['ticket_queue'] == 'Dismissed')
count_by_state[0,2] = np.count_nonzero(df_CCPT16.loc[mask_instate]['ticket_queue'] == 'Notice') 
count_by_state[0,3] = df_CCPT16.loc[mask_instate]['ticket_queue'].size - np.sum(count_by_state[0,0:3])
                
mask_outstate = (df_CCPT16['license_plate_state'] != 'IL')
count_by_state[1,0] = np.count_nonzero(df_CCPT16.loc[mask_outstate]['ticket_queue'] == 'Paid')
count_by_state[1,1] = np.count_nonzero(df_CCPT16.loc[mask_outstate]['ticket_queue'] == 'Dismissed')
count_by_state[1,2] = np.count_nonzero(df_CCPT16.loc[mask_outstate]['ticket_queue'] == 'Notice') 
count_by_state[1,3] = df_CCPT16.loc[mask_outstate]['ticket_queue'].size - np.sum(count_by_state[1,0:3])

In [ ]:
from matplotlib import rc

plt.figure(figsize=(9,9))
plt.tick_params(labelbottom=True)
rc('font',weight = 'bold')
r = np.arange(2)
# bar width
BW = 1

plt.bar(r, count_by_state[:,0], color='blue', edgecolor='white', width=BW,label = 'Paid')
plt.bar(r, count_by_state[:,1], bottom=count_by_state[:,0], color='red', edgecolor='white', width=BW,label='Dismissed')
plt.bar(r, count_by_state[:,2], bottom=np.sum(count_by_state[:,0:2],axis = 1), color='green', edgecolor='white', width=BW,label='Likely Guilty')
plt.bar(r, count_by_state[:,3], bottom=np.sum(count_by_state[:,0:3],axis = 1), color='magenta', edgecolor='white', width=BW,label='Unlikely to Pay')

plt.xticks(r, ['In state plate','Out of state plate'], fontweight='bold',rotation='vertical')
#plt.xlabel("Violation Type (Top 20)")
plt.ylabel("No. of Occurance between 2016 and 2017")
plt.legend(fontsize = 16)
plt.show()

In [ ]:
plt.figure(figsize=(18,9))
df_CCPT16['violation_description'].value_counts().head(30).plot(kind='pie')
#plt.tick_params(labelbottom='on')
plt.ylabel('', fontsize=16);
plt.xlabel('Violation Type', fontsize=16);
plt.title('Number of Parking Tickets Given for Each Violation Code', fontsize=16);

In [ ]:
violation_type = df_CCPT16['violation_description'].value_counts().index
ticket_status = df_CCPT16['ticket_queue'].value_counts().index
# Here I categorize the ticket status in to 4:
# Paid, Dissmissed, Likely to be Guilty (Notice), Unlikely to pay (the rest: Define, Court, Bankrupcy,Hearing Req)
count_by_vt = np.zeros([20,4])

for vt_index in np.arange(20):
    mask = (df_CCPT16['violation_description'] == violation_type[vt_index])
    count_by_vt[vt_index,0] = np.count_nonzero(df_CCPT16.loc[mask]['ticket_queue'] == 'Paid')
    count_by_vt[vt_index,1] = np.count_nonzero(df_CCPT16.loc[mask]['ticket_queue'] == 'Dismissed')
    count_by_vt[vt_index,2] = np.count_nonzero(df_CCPT16.loc[mask]['ticket_queue'] == 'Notice') 
    count_by_vt[vt_index,3] = df_CCPT16.loc[mask]['ticket_queue'].size - np.sum(count_by_vt[vt_index,0:3])

In [ ]:
plt.figure(figsize=(18,9))
plt.tick_params(labelbottom=True)
rc('font',weight = 'bold')
r = np.arange(20)
# bar width
BW = 1

plt.bar(r, count_by_vt[:,0], color='blue', edgecolor='white', width=BW,label = 'Paid')
plt.bar(r, count_by_vt[:,1], bottom=count_by_vt[:,0], color='red', edgecolor='white', width=BW,label='Dismissed')
plt.bar(r, count_by_vt[:,2], bottom=np.sum(count_by_vt[:,0:2],axis = 1), color='green', edgecolor='white', width=BW,label='Likely Guilty')
plt.bar(r, count_by_vt[:,3], bottom=np.sum(count_by_vt[:,0:3],axis = 1), color='magenta', edgecolor='white', width=BW,label='Unlikely to Pay')

plt.xticks(r, violation_type[:20], fontweight='bold',rotation='vertical')
plt.xlabel("Violation Type (Top 20)")
plt.ylabel("No. of Occurance between 2016 and 2017")
plt.legend(fontsize = 16)
plt.show()

In [ ]:
hd = {'Unique Entry': df_CCPT.nunique(axis = 0),
        'Nan Entry': df_CCPT.isnull().any()}
pd.DataFrame(data = hd, index = df_CCPT16.columns.values)


In [ ]:
fine_amount = df_CCPT16[['fine_level1_amount','fine_level2_amount','current_amount_due','total_payments']]

In [ ]:
# fine_level2 is always twice the fine_level1, which makes it not helpful here
np.mean(fine_amount['fine_level2_amount']//fine_amount['fine_level1_amount'])

In [ ]:
fine_amount['final_amount'] = fine_amount['current_amount_due'] + fine_amount['total_payments']
fine_amount['due_or_not'] = fine_amount['current_amount_due'] > 0

In [ ]:
sns.set()
#ax = sns.scatterplot(x=fine_amount['fine_level1_amount'], y = fine_amount['final_amount'], hue=fine_amount['due_or_not'],data=tips)

ax = sns.jointplot(x = 'fine_level1_amount', y = 'final_amount',data=fine_amount[:10000])
plt.show()

In [ ]:
fine_level = np.sort(df_CCPT16['fine_level1_amount'].unique())
fine_due_or_not = np.zeros([np.size(fine_level),2])

for fl_indx in np.arange(np.size(fine_level)):
    mask = (df_CCPT16['fine_level1_amount'] == fine_level[fl_indx])
    fine_due_or_not[fl_indx,0] = np.count_nonzero(df_CCPT16.loc[mask]['current_amount_due'] <= 0 ) # paid
    fine_due_or_not[fl_indx,1] = np.count_nonzero(df_CCPT16.loc[mask]['current_amount_due'] > 0 ) # due

In [ ]:
plt.figure(figsize=(18,9))
plt.tick_params(labelbottom=True)
rc('font',weight = 'bold')
r = np.arange(np.size(fine_level))
# bar width
BW = 1

plt.bar(r, fine_due_or_not[:,0], color='blue', edgecolor='white', width=BW,label = 'Paid')
plt.bar(r, fine_due_or_not[:,1], bottom=fine_due_or_not[:,0], color='red', edgecolor='white', width=BW,label='Due')

plt.xticks(r, fine_level, fontweight='bold',rotation='vertical')
plt.xlabel("Initial Fine Amount")
plt.ylabel("No. of Ticket between 2016 and 2017")
plt.legend(fontsize = 16)
plt.show()



In [ ]:
 np.sort(df_CCPT16['fine_level1_amount'].unique())

In [ ]:
df_CCPT16['address'].head()

In [ ]:
address_df = pd.DataFrame(df_CCPT16['address'].str.split(', ').tolist(), columns=['address','city','state','None'])
#adderss_df.drop(columns = 'None',inplace = True)

In [ ]:
address_df.head()

In [ ]:
address_df['city'].value_counts()

In [ ]:
mask = (address_df['state']==' chicago')
address_df.loc[mask]

In [ ]:
# found anomoly here, change it
mask_add = (df_CCPT16['address'] == '600 n st, louis, chicago, il')
# I know this looks ugly, I haven't found a better way :(
df_CCPT16['address'] = df_CCPT16['address'].str.replace('600 n st, louis, chicago, il','600 n st louis, chicago, il')

In [ ]:
df_CCPT16.loc[mask_add]['address']

In [ ]:
address_df = pd.DataFrame(df_CCPT16['address'].str.split(', ').tolist(), columns=['address','city','state'])

In [ ]:
address_df['city'].value_counts()
# now the address should have been cleaned

In [ ]:
address_df['state'].unique()
print('Number of unique addresses: {}'.format(len(address_df['address'].unique())))

In [ ]:
address_df = pd.DataFrame(address_df['address'].unique(), columns=['address'])
address_df['lat'] = np.nan
address_df['lng'] = np.nan
address_df.head()

In [ ]:
import geocoder
import requests
import time

In [ ]:
class GeoSessions:
    def __init__(self):
        self.Arcgis = requests.Session()
        self.Komoot = requests.Session()
        self.USCensus = requests.Session()
        self.OSM = requests.Session()

def create_sessions():
    return GeoSessions()

def geocode_address(address, s):
    g = geocoder.komoot(address, session=s.Komoot)
    if (g.ok == False):        
        g = geocoder.osm(address, session=s.OSM)
    if (g.ok == False):
        g = geocoder.arcgis(address, session=s.Arcgis)
    if (g.ok == False):
        g = geocoder.uscensus(address, session=s.USCensus)
    return g

In [ ]:
s = create_sessions()


In [ ]:
addr = address_df['address'][0] + ', Chicago, IL'
tmp = geocode_address(addr, s)
print(addr)
print('Latitude:  {}'.format(tmp.lat))
print('Longitude: {}'.format(tmp.lng))

In [ ]:
failed_inds = []
iter_between_saves = 100

In [ ]:
missing_latlon_ind = address_df[address_df['lat'].isnull()][:60001].index.tolist()

for i in missing_latlon_ind:
    try:
        if i%iter_between_saves == 0:
            print('reached index {}, saving df'.format(i))
            address_df.to_csv('CCPT_frac_1617.csv', index=False)
        addr = address_df['address'][i] + ', Chicago, IL'
        tmp = geocode_address(addr, s)
        address_df.loc[i,'lat'] = tmp.lat
        address_df.loc[i,'lng'] = tmp.lng
    except Exception as e:
        failed_inds.append(i)
address_df.to_csv('CCPT_frac_1617.csv', index=False)

In [ ]:
address_df[address_df['lat'].isnull()]

In [10]:
# load the presaved df for geocoded df
df_geoloc = pd.read_csv('./data/Geocoded_CCPT_1617.csv')
df_geoloc['address'] = df_geoloc['address'] + ', chicago, il'


In [11]:
geocoded_df = pd.merge(left=df_CCPT16, right=df_geoloc, how='inner', on='address')
geocoded_df.head()

,issue_date,license_plate_state,license_plate_type,violation_code,violation_description,vehicle_make,fine_level1_amount,fine_level2_amount,current_amount_due,total_payments,ticket_queue,ticket_queue_date,notice_level,hearing_disposition,officer,address,lat,lng
0,2016-05-19 17:40:00,IL,PAS,0976160A,REAR AND FRONT PLATE REQUIRED,LNDR,60,120,60.0,0.0,Define,2016-05-25 00:00:00,NaN,NaN,05238,"500 e 46, chicago, il",41.834909,-87.614170
1,2016-01-30 00:34:00,IL,TMP,0976160F,EXPIRED PLATES OR TEMPORARY REGISTRATION,CHRY,60,120,146.4,0.0,Notice,2016-02-11 00:00:00,FINL,NaN,10109,"500 e 46, chicago, il",41.834909,-87.614170
2,2016-11-05 20:35:00,IL,PAS,0964100C,BLOCK ACCESS/ALLEY/DRIVEWAY/FIRELANE,HOND,150,300,0.0,150.0,Paid,2016-11-15 00:00:00,NaN,NaN,5121,"6100 s archer ave, chicago, il",41.794587,-87.770287
3,2016-05-27 17:30:00,IL,PAS,0964050J,DISABLED PARKING ZONE,FORD,250,500,0.0,250.0,Paid,2016-06-07 00:00:00,NaN,NaN,5652,"6100 s archer ave, chicago, il",41.794587,-87.770287
4,2017-11-24 16:23:00,IL,PAS,0964100C,BLOCK ACCESS/ALLEY/DRIVEWAY/FIRELANE,DODG,150,300,300.0,0.0,Notice,2017-12-21 00:00:00,FINL,NaN,5652,"6100 s archer ave, chicago, il",41.794587,-87.770287


In [12]:
mask_chi = ((geocoded_df['lat']>=41.63) & (geocoded_df['lat']<= 42.05) &
               (geocoded_df['lng']>= -88.0) & (geocoded_df['lng']<= -87.5))

geocoded_df = geocoded_df.loc[mask_chi]

In [13]:
import folium
import folium.plugins as plugins

#from folium.plugins import HeatMap

In [14]:
chicago_lat = 41.8
chicago_lng = -87.7

In [15]:
df_permit = geocoded_df.loc[geocoded_df['violation_description'] == 'RESIDENTIAL PERMIT PARKING']
df_permit.head()
#df_CCPT16['issue_date'].groupby(df_CCPT16['issue_date'].dt.hour).count().plot(kind='bar')

,issue_date,license_plate_state,license_plate_type,violation_code,violation_description,vehicle_make,fine_level1_amount,fine_level2_amount,current_amount_due,total_payments,ticket_queue,ticket_queue_date,notice_level,hearing_disposition,officer,address,lat,lng
64,2017-11-17 18:07:00,IL,PAS,0964090E,RESIDENTIAL PERMIT PARKING,TOYT,75,150,0.0,0.0,Dismissed,2018-01-11 00:00:00,VIOL,Not Liable,11371,"2200 n st louis, chicago, il",41.928285,-87.714662
720,2017-12-28 00:47:00,IL,PAS,0964090E,RESIDENTIAL PERMIT PARKING,VOLK,75,150,0.0,75.0,Paid,2018-01-04 00:00:00,NaN,NaN,1640,"800 s oakley boulevard, chicago, il",41.871201,-87.684071
721,2016-02-12 00:45:00,IL,PAS,0964090E,RESIDENTIAL PERMIT PARKING,NISS,75,150,0.0,150.0,Paid,2016-05-05 00:00:00,FINL,NaN,1462,"800 s oakley boulevard, chicago, il",41.871201,-87.684071
722,2016-08-16 20:26:00,IL,PAS,0964090E,RESIDENTIAL PERMIT PARKING,DODG,75,150,0.0,75.0,Paid,2016-08-25 00:00:00,NaN,NaN,1125,"1700 w cornelia, chicago, il",41.943056,-87.813918
725,2016-10-07 17:37:00,IL,PAS,0964090E,RESIDENTIAL PERMIT PARKING,PONT,75,150,0.0,75.0,Paid,2016-11-07 00:00:00,VIOL,NaN,767,"1700 w cornelia, chicago, il",41.943056,-87.813918


In [16]:
df_permit_hm = df_permit[['lat','lng']]

In [17]:
# There are quite some outliers, here I will just remove the ones outside Chicago, 
# later I will do the more careful cleaning

mask_chi = ((df_permit['lat']>=41.63) & (df_permit['lat']<= 42.05) &
               (df_permit['lng']>= -88.0) & (df_permit['lng']<= -87.5))
df_permit_hm = df_permit_hm.loc[mask_chi]

In [18]:
permit_data = [[row['lat'],row['lng']] for index, row in df_permit_hm.iterrows()]

In [19]:
np.size(permit_data)

150956

In [20]:
m = folium.Map([chicago_lat,chicago_lng],  tiles = "Stamen Toner",zoom_start=10)
hm = plugins.HeatMap(permit_data,radius = 9)
hm.add_to(m)
m

In [ ]:
df_permit['Weight'] = df_permit['issue_date'].dt.hour
df_permit['Weight'] = df_permit['Weight'].astype(float)
df_permit_hmt = df_permit[['lat','lng','Weight']]

In [ ]:
permit_timeseries = [[[row['lat'],row['lng']] 
                           for index, row in df_permit_hmt[df_permit_hmt['Weight'] == i].iterrows()] for i in range(0,24)]

In [ ]:
hours_indx = [(datetime.time(i).strftime('%I %p')) for i in range(24)]
mt = folium.Map([chicago_lat,chicago_lng],tiles = "Stamen Toner",zoom_start=10)
hmt = plugins.HeatMapWithTime(permit_timeseries,auto_play=True,radius = 5, min_opacity = 1,index = hours_indx)
hmt.add_to(mt)
mt

In [ ]:
plt.figure(figsize=(16,8))
df_permit['issue_date'].groupby(df_permit['issue_date'].dt.hour).count().plot(kind='bar')
plt.tick_params(labelbottom='on')
plt.ylabel('No. of cars', fontsize=16);
plt.xlabel('Day Time', fontsize=16);
plt.title('Parking ticket issued at different time of the day', fontsize=16);


In [ ]:
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [21]:
geocoded_df.head()

,issue_date,license_plate_state,license_plate_type,violation_code,violation_description,vehicle_make,fine_level1_amount,fine_level2_amount,current_amount_due,total_payments,ticket_queue,ticket_queue_date,notice_level,hearing_disposition,officer,address,lat,lng
0,2016-05-19 17:40:00,IL,PAS,0976160A,REAR AND FRONT PLATE REQUIRED,LNDR,60,120,60.0,0.0,Define,2016-05-25 00:00:00,NaN,NaN,05238,"500 e 46, chicago, il",41.834909,-87.614170
1,2016-01-30 00:34:00,IL,TMP,0976160F,EXPIRED PLATES OR TEMPORARY REGISTRATION,CHRY,60,120,146.4,0.0,Notice,2016-02-11 00:00:00,FINL,NaN,10109,"500 e 46, chicago, il",41.834909,-87.614170
2,2016-11-05 20:35:00,IL,PAS,0964100C,BLOCK ACCESS/ALLEY/DRIVEWAY/FIRELANE,HOND,150,300,0.0,150.0,Paid,2016-11-15 00:00:00,NaN,NaN,5121,"6100 s archer ave, chicago, il",41.794587,-87.770287
3,2016-05-27 17:30:00,IL,PAS,0964050J,DISABLED PARKING ZONE,FORD,250,500,0.0,250.0,Paid,2016-06-07 00:00:00,NaN,NaN,5652,"6100 s archer ave, chicago, il",41.794587,-87.770287
4,2017-11-24 16:23:00,IL,PAS,0964100C,BLOCK ACCESS/ALLEY/DRIVEWAY/FIRELANE,DODG,150,300,300.0,0.0,Notice,2017-12-21 00:00:00,FINL,NaN,5652,"6100 s archer ave, chicago, il",41.794587,-87.770287


In [22]:
to_drop = ['fine_level2_amount','current_amount_due','total_payments',
          'ticket_queue_date','notice_level','hearing_disposition','address']
geocoded_df.drop(labels=to_drop ,axis = 1,inplace = True)

In [23]:
row2drop= ['license_plate_type']
geocoded_df.dropna(axis = 0, how = 'any', subset = row2drop, inplace = True)
#geocoded_df['license_plate_type'].value_counts()

In [ ]:
hd = {'Unique Entry': geocoded_df.nunique(axis = 0),
        'Nan Entry': geocoded_df.isnull().any()}
pd.DataFrame(data = hd, index = geocoded_df.columns.values)

In [ ]:
geocoded_df['ticket_queue'].value_counts()

In [24]:
geocoded_df['Outcome'] = np.nan
geocoded_df['Outcome'] = geocoded_df['ticket_queue'].map( {'Paid': 0, 'Dismissed': 1, 'Notice': 2,'Define':3,'Bankruptcy':3,'Court':3,'Hearing Req':3} ).astype(int)
#geocoded_df.loc[geocoded_df['ticket_queue']=='Court']

In [25]:
wd = {'Month':geocoded_df['issue_date'].dt.month,
      'Weekday': geocoded_df['issue_date'].dt.weekday,
      'Hour': geocoded_df['issue_date'].dt.hour}
df_date = pd.DataFrame(data = wd, index = geocoded_df.index)

In [26]:
df_ml = pd.concat([geocoded_df,df_date],axis=1)

In [ ]:
df_ml.head()

In [27]:
mask_in_state = (df_ml['license_plate_state'] == 'IL')
df_ml['Plate_State'] = np.zeros(df_ml['license_plate_state'].count()).astype(int)
df_ml.loc[mask_in_state,'Plate_State'] = np.ones(df_ml.loc[mask_in_state]['Plate_State'].count()).astype(int)

In [ ]:
df_ml['license_plate_type'].value_counts()

In [28]:
mask_plate_type = df_ml['license_plate_type'].isin(['PAS','TRK','TMP','TXI','DLC','FFM'])
df_ml.loc[~mask_plate_type,'license_plate_type'] = 'OTH'

In [29]:
df_ml.loc[~mask_plate_type]

,issue_date,license_plate_state,license_plate_type,violation_code,violation_description,vehicle_make,fine_level1_amount,ticket_queue,officer,lat,lng,Outcome,Month,Weekday,Hour,Plate_State
163,2016-03-02 13:52:00,IL,OTH,0964190A,EXP. METER NON-CENTRAL BUSINESS DISTRICT,TOYT,50,Paid,805,41.857914,-87.663931,0,3,2,13,1
204,2016-01-25 13:25:00,IL,OTH,0976160F,EXPIRED PLATES OR TEMPORARY REGISTRATION,HYUN,60,Paid,800,41.857914,-87.663931,0,1,0,13,1
239,2017-09-13 18:35:00,IL,OTH,0964190A,EXP. METER NON-CENTRAL BUSINESS DISTRICT,CHEV,50,Define,1606,41.857914,-87.663931,3,9,2,18,1
252,2017-06-30 20:34:00,IL,OTH,0976160A,REAR AND FRONT PLATE REQUIRED,JEEP,60,Paid,1623,41.857914,-87.663931,0,6,4,20,1
272,2016-09-07 20:05:00,IL,OTH,0976160F,EXPIRED PLATES OR TEMPORARY REGISTRATION,VESP,60,Paid,1565,41.857914,-87.663931,0,9,2,20,1
366,2016-04-13 12:15:00,IL,OTH,0964190A,EXP. METER NON-CENTRAL BUSINESS DISTRICT,CADI,50,Paid,805,41.857914,-87.663931,0,4,2,12,1
454,2017-09-21 21:55:00,IL,OTH,0964190A,EXP. METER NON-CENTRAL BUSINESS DISTRICT,CADI,50,Paid,830,41.857914,-87.663931,0,9,3,21,1
466,2017-02-04 14:18:00,IL,OTH,0964190A,EXP. METER NON-CENTRAL BUSINESS DISTRICT,HOND,50,Dismissed,1606,41.857914,-87.663931,1,2,5,14,1
514,2016-09-07 20:06:00,IL,OTH,0964190A,EXP. METER NON-CENTRAL BUSINESS DISTRICT,VESP,50,Paid,1565,41.857914,-87.663931,0,9,2,20,1
521,2017-05-05 13:04:00,IL,OTH,0976160A,REAR AND FRONT PLATE REQUIRED,JEEP,60,Paid,791,41.857914,-87.663931,0,5,4,13,1


In [30]:
df_ml['license_plate_type'].value_counts()

PAS    728753
TRK     40702
TMP     33151
OTH     25072
TXI      9464
DLC      4484
FFM      3365
Name: license_plate_type, dtype: int64

In [ ]:
df_ml['vehicle_make'].value_counts()

In [31]:
violation_type = df_ml['violation_description'].value_counts().index
mask_vio_type = df_ml['violation_description'].isin(violation_type[:25])
df_ml.loc[~mask_vio_type,'violation_description'] = 'OTHER'

In [32]:
df_ml.loc[~mask_vio_type]

,issue_date,license_plate_state,license_plate_type,violation_code,violation_description,vehicle_make,fine_level1_amount,ticket_queue,officer,lat,lng,Outcome,Month,Weekday,Hour,Plate_State
11,2016-06-14 11:55:00,IL,PAS,0964160B,OTHER,DODG,60,Define,16396,41.953806,-87.732667,3,6,1,11,1
23,2016-06-13 13:03:00,IL,TXI,0964160B,OTHER,TOYT,60,Paid,16396,41.953806,-87.732667,0,6,0,13,1
51,2017-09-27 08:05:00,IL,PAS,0976140B,OTHER,GMC,100,Define,287,41.928285,-87.714662,3,9,2,8,1
113,2017-02-04 21:25:00,IL,PAS,0964130B,OTHER,HOND,150,Paid,8564,41.857914,-87.663931,0,2,5,21,1
316,2016-03-15 14:36:00,IL,TRK,0940060,OTHER,TOYT,150,Notice,805,41.857914,-87.663931,2,3,1,14,1
362,2016-12-12 23:10:00,IL,PAS,0980120B,OTHER,SATR,25,Define,6060,41.857914,-87.663931,3,12,0,23,1
398,2016-04-11 13:10:00,IL,PAS,0940060,OTHER,ACUR,150,Paid,805,41.857914,-87.663931,0,4,0,13,1
763,2017-03-01 14:59:00,IL,PAS,0964020A,OTHER,JEEP,25,Paid,767,41.943056,-87.813918,0,3,2,14,1
786,2017-08-27 12:06:00,IL,PAS,0980110B,OTHER,NISS,75,Dismissed,684,41.689837,-87.605656,1,8,6,12,1
826,2017-06-06 13:13:00,IL,TRK,0964130,OTHER,FORD,150,Paid,798,41.884899,-87.630473,0,6,1,13,1


In [33]:
 df_ml['violation_description'].value_counts()

EXPIRED PLATES OR TEMPORARY REGISTRATION              119408
STREET CLEANING                                       102610
EXP. METER NON-CENTRAL BUSINESS DISTRICT               95059
NO CITY STICKER VEHICLE UNDER/EQUAL TO 16,000 LBS.     84112
RESIDENTIAL PERMIT PARKING                             75478
PARKING/STANDING PROHIBITED ANYTIME                    58473
EXPIRED METER CENTRAL BUSINESS DISTRICT                49918
OTHER                                                  45977
REAR AND FRONT PLATE REQUIRED                          34080
NO STANDING/PARKING TIME RESTRICTED                    29234
RUSH HOUR PARKING                                      24371
EXPIRED PLATE OR TEMPORARY REGISTRATION                24236
WITHIN 15' OF FIRE HYDRANT                             16463
PARK OR STAND IN BUS/TAXI/CARRIAGE STAND               13678
MISSING/NONCOMPLIANT FRONT AND/OR REAR PLATE            8434
STOP SIGN OR TRAFFIC SIGNAL                             8131
OBSTRUCT ROADWAY        

In [34]:
hd = {'Unique Entry': df_ml.nunique(axis = 0),
        'Nan Entry': df_ml.isnull().any()}
pd.DataFrame(data = hd, index = df_ml.columns.values)

,Unique Entry,Nan Entry
issue_date,486704,False
license_plate_state,62,False
license_plate_type,7,False
violation_code,111,False
violation_description,26,False
vehicle_make,149,False
fine_level1_amount,14,False
ticket_queue,7,False
officer,10530,False
lat,14753,False


In [35]:
to_drop = ['issue_date','license_plate_state','violation_code','ticket_queue']
df_ml.drop(labels=to_drop ,axis = 1,inplace = True)

In [36]:
df_ml.columns

Index(['license_plate_type', 'violation_description', 'vehicle_make',
       'fine_level1_amount', 'officer', 'lat', 'lng', 'Outcome', 'Month',
       'Weekday', 'Hour', 'Plate_State'],
      dtype='object')

In [ ]:
df_ml.officer.value_counts()

In [41]:
df_ml.to_csv('CCPT_ML.csv', index=False)

In [2]:
import h2o
from h2o.estimators import H2ORandomForestEstimator

In [42]:
data = h2o.import_file('CCPT_ML.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [51]:
data['Outcome'].

Outcome
3
2
0
0
2
0
2
2
0
0


In [46]:
train_col = ['license_plate_type','violation_description','fine_level1_amount',
             'Plate_State','Month', 'Weekday', 'Hour']
response_col = ['Outcome']

In [39]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_152-release"; OpenJDK Runtime Environment (build 1.8.0_152-release-1056-b12); OpenJDK 64-Bit Server VM (build 25.152-b12, mixed mode)
  Starting server from /home/chao/anaconda3/h2o_jar/h2o.jar
  Ice root: /tmp/tmposs3dlii
  JVM stdout: /tmp/tmposs3dlii/h2o_chao_started_from_python.out
  JVM stderr: /tmp/tmposs3dlii/h2o_chao_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,8 months and 2 days !!!
H2O cluster name:,H2O_from_python_chao_5jhzqz
H2O cluster total nodes:,1
H2O cluster free memory:,3.471 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [47]:
train, test = data.split_frame(ratios=[0.8])

In [48]:
model = H2ORandomForestEstimator(ntrees=50, max_depth=20, nfolds=10)

In [54]:
model.train(x=train_col, y=response_col, training_frame=train)

Outcome
3
2
0
0
2
0
2
2
0
0


Outcome
3
2
0
0
2
0
2
2
0
0


H2OTypeError: Argument `y` should be a None | integer | string, got H2OFrame 

Outcome
3
2
0
0
2
0
2
2
0
0
